In [17]:
import pandas as pd
from tabulate import tabulate


def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"


def get_stat_df(result_df, verbose=True, metrics=['micro_auprc', 'micro_auroc',  'mcc',
                                                  'sensitivity', 'precision']):
    # Sample loading of your CSV - this would be replaced with your actual loading

    grouped = result_df.groupby('model_key').agg({
        k: ['mean', 'std'] for k in metrics
    })

    record_counts = result_df.groupby('model_key').size()

    for metric in metrics:
        grouped[f'{metric}_formatted'] = grouped.apply(
            lambda row: format_mean_std(row, metric), axis=1)

    # Add the record count column to the grouped dataframe
    grouped['record_count'] = record_counts.values

    formatted_df = grouped[[
        k + '_formatted' for k in metrics] + ['record_count']]
    formatted_df.columns = metrics + ['record_count']
    formatted_df = formatted_df.sort_values(metrics[0], ascending=True)

    formatted_df = formatted_df.reset_index()

    if verbose:
        markdown_table = tabulate(formatted_df[[
                                  'model_key'] + metrics], headers='keys', tablefmt='pipe', showindex=False)
        print(markdown_table)

    return formatted_df

df_imatinib = pd.read_csv('imatinib_stats.csv')

def global_filter(df):
    return df[
        (df['model_key'].str.contains(r'^esm-t33-gearnet-(?!\d)') & 
        ~df['model_key'].str.contains(r'r50') &
         ~df['model_key'].str.contains(r'r80') &
         ~df['model_key'].str.contains(r'r10') &
         ~df['model_key'].str.contains(r'rus') &
         ~df['model_key'].str.contains(r'ensemble')) | (df['model_key'] == 'esm-t33-gearnet') | (df['model_key'] == 'esm-t33') | (df['model_key'] == 'esm-t33-pretrained')
    ]

get_stat_df(global_filter(df_imatinib))

| model_key                                | micro_auprc   | micro_auroc   | mcc           | sensitivity   | precision     |
|:-----------------------------------------|:--------------|:--------------|:--------------|:--------------|:--------------|
| esm-t33-pretrained                       | 0.502 ± 0.017 | 0.857 ± 0.015 | 0.579 ± 0.033 | 0.548 ± 0.068 | 0.663 ± 0.026 |
| esm-t33                                  | 0.511 ± 0.021 | 0.864 ± 0.033 | 0.612 ± 0.026 | 0.594 ± 0.032 | 0.678 ± 0.052 |
| esm-t33-gearnet                          | 0.581 ± 0.027 | 0.846 ± 0.010 | 0.548 ± 0.077 | 0.513 ± 0.046 | 0.656 ± 0.172 |
| esm-t33-gearnet-pretrained               | 0.602 ± 0.036 | 0.820 ± 0.044 | 0.627 ± 0.020 | 0.533 ± 0.035 | 0.782 ± 0.063 |
| esm-t33-gearnet-resiboost-r90            | 0.638 ± 0.018 | 0.845 ± 0.010 | 0.610 ± 0.040 | 0.548 ± 0.068 | 0.741 ± 0.134 |
| esm-t33-gearnet-adaboost-r90             | 0.640 ± 0.017 | 0.847 ± 0.012 | 0.610 ± 0.058 | 0.555 ± 0.083 | 0.741 ± 0.170 |


,model_key,micro_auprc,micro_auroc,mcc,sensitivity,precision,record_count
0,esm-t33-pretrained,0.502 ± 0.017,0.857 ± 0.015,0.579 ± 0.033,0.548 ± 0.068,0.663 ± 0.026,5
1,esm-t33,0.511 ± 0.021,0.864 ± 0.033,0.612 ± 0.026,0.594 ± 0.032,0.678 ± 0.052,5
2,esm-t33-gearnet,0.581 ± 0.027,0.846 ± 0.010,0.548 ± 0.077,0.513 ± 0.046,0.656 ± 0.172,5
3,esm-t33-gearnet-pretrained,0.602 ± 0.036,0.820 ± 0.044,0.627 ± 0.020,0.533 ± 0.035,0.782 ± 0.063,5
4,esm-t33-gearnet-resiboost-r90,0.638 ± 0.018,0.845 ± 0.010,0.610 ± 0.040,0.548 ± 0.068,0.741 ± 0.134,10
5,esm-t33-gearnet-adaboost-r90,0.640 ± 0.017,0.847 ± 0.012,0.610 ± 0.058,0.555 ± 0.083,0.741 ± 0.170,10
6,esm-t33-gearnet-pretrained-adaboost-r90,0.649 ± 0.021,0.844 ± 0.030,0.615 ± 0.017,0.565 ± 0.075,0.724 ± 0.106,5
7,esm-t33-gearnet-pretrained-resiboost-r90,0.656 ± 0.027,0.853 ± 0.039,0.624 ± 0.014,0.554 ± 0.075,0.756 ± 0.100,5


In [18]:
df_dasatinib = pd.read_csv('dasatinib_stats.csv')
get_stat_df(global_filter(df_dasatinib))

| model_key                                | micro_auprc   | micro_auroc   | mcc           | sensitivity   | precision     |
|:-----------------------------------------|:--------------|:--------------|:--------------|:--------------|:--------------|
| esm-t33-gearnet                          | 0.612 ± 0.093 | 0.905 ± 0.030 | 0.588 ± 0.076 | 0.508 ± 0.083 | 0.730 ± 0.068 |
| esm-t33-pretrained                       | 0.617 ± 0.035 | 0.907 ± 0.014 | 0.584 ± 0.028 | 0.576 ± 0.056 | 0.647 ± 0.090 |
| esm-t33                                  | 0.624 ± 0.050 | 0.900 ± 0.040 | 0.561 ± 0.030 | 0.515 ± 0.074 | 0.669 ± 0.070 |
| esm-t33-gearnet-pretrained               | 0.628 ± 0.045 | 0.904 ± 0.017 | 0.562 ± 0.076 | 0.393 ± 0.119 | 0.876 ± 0.093 |
| esm-t33-gearnet-adaboost-r90             | 0.694 ± 0.049 | 0.935 ± 0.016 | 0.567 ± 0.116 | 0.420 ± 0.168 | 0.843 ± 0.071 |
| esm-t33-gearnet-resiboost-r90            | 0.696 ± 0.059 | 0.936 ± 0.009 | 0.587 ± 0.080 | 0.485 ± 0.134 | 0.774 ± 0.072 |


,model_key,micro_auprc,micro_auroc,mcc,sensitivity,precision,record_count
0,esm-t33-gearnet,0.612 ± 0.093,0.905 ± 0.030,0.588 ± 0.076,0.508 ± 0.083,0.730 ± 0.068,5
1,esm-t33-pretrained,0.617 ± 0.035,0.907 ± 0.014,0.584 ± 0.028,0.576 ± 0.056,0.647 ± 0.090,5
2,esm-t33,0.624 ± 0.050,0.900 ± 0.040,0.561 ± 0.030,0.515 ± 0.074,0.669 ± 0.070,5
3,esm-t33-gearnet-pretrained,0.628 ± 0.045,0.904 ± 0.017,0.562 ± 0.076,0.393 ± 0.119,0.876 ± 0.093,5
4,esm-t33-gearnet-adaboost-r90,0.694 ± 0.049,0.935 ± 0.016,0.567 ± 0.116,0.420 ± 0.168,0.843 ± 0.071,5
5,esm-t33-gearnet-resiboost-r90,0.696 ± 0.059,0.936 ± 0.009,0.587 ± 0.080,0.485 ± 0.134,0.774 ± 0.072,5
6,esm-t33-gearnet-pretrained-adaboost-r90,0.712 ± 0.038,0.941 ± 0.012,0.590 ± 0.050,0.488 ± 0.145,0.787 ± 0.111,5
7,esm-t33-gearnet-pretrained-resiboost-r90,0.715 ± 0.035,0.941 ± 0.012,0.606 ± 0.053,0.475 ± 0.097,0.828 ± 0.058,5


In [19]:
df_bosutinib = pd.read_csv('bosutinib_stats.csv')
get_stat_df(global_filter(df_bosutinib))

| model_key                                | micro_auprc   | micro_auroc   | mcc           | sensitivity   | precision     |
|:-----------------------------------------|:--------------|:--------------|:--------------|:--------------|:--------------|
| esm-t33-gearnet-resiboost-r90            | 0.806 ± 0.042 | 0.942 ± 0.021 | 0.731 ± 0.047 | 0.697 ± 0.063 | 0.814 ± 0.130 |
| esm-t33-gearnet-adaboost-r90             | 0.809 ± 0.039 | 0.947 ± 0.020 | 0.739 ± 0.034 | 0.697 ± 0.092 | 0.831 ± 0.107 |
| esm-t33-gearnet                          | 0.811 ± 0.034 | 0.953 ± 0.007 | 0.717 ± 0.027 | 0.710 ± 0.043 | 0.768 ± 0.082 |
| esm-t33-gearnet-pretrained               | 0.830 ± 0.010 | 0.957 ± 0.012 | 0.735 ± 0.019 | 0.677 ± 0.058 | 0.840 ± 0.062 |
| esm-t33-pretrained                       | 0.832 ± 0.040 | 0.936 ± 0.035 | 0.754 ± 0.047 | 0.660 ± 0.084 | 0.897 ± 0.014 |
| esm-t33                                  | 0.852 ± 0.020 | 0.952 ± 0.018 | 0.745 ± 0.030 | 0.667 ± 0.059 | 0.871 ± 0.059 |


,model_key,micro_auprc,micro_auroc,mcc,sensitivity,precision,record_count
0,esm-t33-gearnet-resiboost-r90,0.806 ± 0.042,0.942 ± 0.021,0.731 ± 0.047,0.697 ± 0.063,0.814 ± 0.130,5
1,esm-t33-gearnet-adaboost-r90,0.809 ± 0.039,0.947 ± 0.020,0.739 ± 0.034,0.697 ± 0.092,0.831 ± 0.107,5
2,esm-t33-gearnet,0.811 ± 0.034,0.953 ± 0.007,0.717 ± 0.027,0.710 ± 0.043,0.768 ± 0.082,5
3,esm-t33-gearnet-pretrained,0.830 ± 0.010,0.957 ± 0.012,0.735 ± 0.019,0.677 ± 0.058,0.840 ± 0.062,5
4,esm-t33-pretrained,0.832 ± 0.040,0.936 ± 0.035,0.754 ± 0.047,0.660 ± 0.084,0.897 ± 0.014,5
5,esm-t33,0.852 ± 0.020,0.952 ± 0.018,0.745 ± 0.030,0.667 ± 0.059,0.871 ± 0.059,5
6,esm-t33-gearnet-pretrained-resiboost-r90,0.872 ± 0.023,0.968 ± 0.009,0.766 ± 0.024,0.730 ± 0.110,0.850 ± 0.116,5
7,esm-t33-gearnet-pretrained-adaboost-r90,0.875 ± 0.026,0.969 ± 0.010,0.765 ± 0.025,0.720 ± 0.114,0.860 ± 0.119,5


In [20]:
def get_stat_df_combined(metrics=['mcc'], verbose=True):
    df_imatinib = get_stat_df(global_filter(pd.read_csv('imatinib_stats.csv')), verbose=False)
    df_dasatinib = get_stat_df(global_filter(pd.read_csv('dasatinib_stats.csv')), verbose=False)
    df_bosutinib = get_stat_df(global_filter(pd.read_csv('bosutinib_stats.csv')), verbose=False)

    df_imatinib = df_imatinib.set_index('model_key')
    df_dasatinib = df_dasatinib.set_index('model_key')
    df_bosutinib = df_bosutinib.set_index('model_key')

    df_combined = pd.concat([df_imatinib, df_dasatinib, df_bosutinib], axis=1, keys=['imatinib', 'dasatinib', 'bosutinib'])
    df_combined = df_combined[[(drug, met) for drug in ['imatinib', 'dasatinib', 'bosutinib'] for met in metrics]]
    
    if verbose:
        markdown_table = tabulate(df_combined, headers='keys', tablefmt='pipe', showindex=True)
        print(markdown_table)

    return df_combined


get_stat_df_combined(metrics=['mcc', 'micro_auprc'])

| model_key                                | ('imatinib', 'mcc')   | ('imatinib', 'micro_auprc')   | ('dasatinib', 'mcc')   | ('dasatinib', 'micro_auprc')   | ('bosutinib', 'mcc')   | ('bosutinib', 'micro_auprc')   |
|:-----------------------------------------|:----------------------|:------------------------------|:-----------------------|:-------------------------------|:-----------------------|:-------------------------------|
| esm-t33-pretrained                       | 0.579 ± 0.033         | 0.502 ± 0.017                 | 0.584 ± 0.028          | 0.617 ± 0.035                  | 0.754 ± 0.047          | 0.832 ± 0.040                  |
| esm-t33                                  | 0.612 ± 0.026         | 0.511 ± 0.021                 | 0.561 ± 0.030          | 0.624 ± 0.050                  | 0.745 ± 0.030          | 0.852 ± 0.020                  |
| esm-t33-gearnet                          | 0.548 ± 0.077         | 0.581 ± 0.027                 | 0.588 ± 0.076          | 0.612 

imatinib                 \
                                                    mcc    micro_auprc   
model_key                                                                
esm-t33-pretrained                        0.579 ± 0.033  0.502 ± 0.017   
esm-t33                                   0.612 ± 0.026  0.511 ± 0.021   
esm-t33-gearnet                           0.548 ± 0.077  0.581 ± 0.027   
esm-t33-gearnet-pretrained                0.627 ± 0.020  0.602 ± 0.036   
esm-t33-gearnet-resiboost-r90             0.610 ± 0.040  0.638 ± 0.018   
esm-t33-gearnet-adaboost-r90              0.610 ± 0.058  0.640 ± 0.017   
esm-t33-gearnet-pretrained-adaboost-r90   0.615 ± 0.017  0.649 ± 0.021   
esm-t33-gearnet-pretrained-resiboost-r90  0.624 ± 0.014  0.656 ± 0.027   

                                              dasatinib                 \
                                                    mcc    micro_auprc   
model_key                                                                
esm-t33-pretrained                        0.584 ± 0.028  0.617 ± 0.035   
esm-t33                                   0.561 ± 0.030  0.624 ± 0.050   
esm-t33-gearnet                           0.588 ± 0.076  0.612 ± 0.093   
esm-t33-gearnet-pretrained                0.562 ± 0.076  0.628 ± 0.045   
esm-t33-gearnet-resiboost-r90             0.587 ± 0.080  0.696 ± 0.059   
esm-t33-gearnet-adaboost-r90              0.567 ± 0.116  0.694 ± 0.049   
esm-t33-gearnet-pretrained-adaboost-r90   0.590 ± 0.050  0.712 ± 0.038   
esm-t33-gearnet-pretrained-resiboost-r90  0.606 ± 0.053  0.715 ± 0.035   

                                              bosutinib                 
                                                    mcc    micro_auprc  
model_key                                                               
esm-t33-pretrained                        0.754 ± 0.047  0.832 ± 0.040  
esm-t33                                   0.745 ± 0.030  0.852 ± 0.020  
esm-t33-gearnet                           0.717 ± 0.027  0.811 ± 0.034  
esm-t33-gearnet-pretrained                0.735 ± 0.019  0.830 ± 0.010  
esm-t33-gearnet-resiboost-r90             0.731 ± 0.047  0.806 ± 0.042  
esm-t33-gearnet-adaboost-r90              0.739 ± 0.034  0.809 ± 0.039  
esm-t33-gearnet-pretrained-adaboost-r90   0.765 ± 0.025  0.875 ± 0.026  
esm-t33-gearnet-pretrained-resiboost-r90  0.766 ± 0.024  0.872 ± 0.023